In [5]:
import numpy as np
from autograd import Variable, LinearLayer
import autograd

from IPython.display import clear_output
training_input_matrix = np.array([[9]])
training_output_vector = np.array([1])

sigmoid = lambda x: np.exp(x)/(np.exp(x)+1)
linear_layer = LinearLayer(2, 1)
linear_layer.matrix.data = np.array([[0.0]], dtype=float)
linear_layer.biases.data = np.array([[0.0]], dtype=float)

debug = False

number_of_epochs = 200
learning_rate = 1e-5
sgd = autograd.optimizer.SGD(learning_rate)

for epoch in range(number_of_epochs):
    for i, (x, y) in enumerate(list(zip(training_input_matrix, training_output_vector))):
        logit = linear_layer(x)
        exp = logit.exp()
        exp_plus_one = exp+1
        y_hat = exp / exp_plus_one
        diff = (y_hat - y)
        loss = diff ** 2
        print(f"loss {loss}")
        if debug:
            print(f"x {repr(x)}")
            print(f"y {repr(y)}")
            print(f"linear_layer.matrix {repr(linear_layer.matrix)}")
            print(f"linear_layer.biases {repr(linear_layer.biases)}")
            print(f"logit {logit}")
            print(f"exp {exp}")
            print(f"exp_plus_one {exp_plus_one}")
            print(f"y_hat {y_hat}")
            print(f"diff {diff}")
            print(f"loss {loss}")
            print()
        var2grad = sgd.take_training_step(loss)
        if debug:
            print('\n'*2)
            print(f"var2grad[linear_layer.matrix] \n{repr(var2grad[linear_layer.matrix])}\n")
            print(f"var2grad[linear_layer.biases] \n{repr(var2grad[linear_layer.biases])}\n")
            print(f"var2grad[logit] \n{repr(var2grad[logit])}\n")
            print(f"var2grad[exp] \n{repr(var2grad[exp])}\n")
            print(f"var2grad[exp_plus_one] \n{repr(var2grad[exp_plus_one])}\n")
            print(f"var2grad[y_hat] \n{repr(var2grad[y_hat])}\n")
            print(f"var2grad[diff] \n{repr(var2grad[diff])}\n")
            print(f"var2grad[loss] \n{repr(var2grad[loss])}\n")
            print('\n'*8)
        if np.isnan(loss.data).any():
            break
        #raise Exception

loss Variable([[0.25]])
loss Variable([[0.24994875]])
loss Variable([[0.24989752]])
loss Variable([[0.24984629]])
loss Variable([[0.24979507]])
loss Variable([[0.24974387]])
loss Variable([[0.24969267]])
loss Variable([[0.24964149]])
loss Variable([[0.24959032]])
loss Variable([[0.24953915]])
loss Variable([[0.249488]])
loss Variable([[0.24943686]])
loss Variable([[0.24938572]])
loss Variable([[0.2493346]])
loss Variable([[0.24928349]])
loss Variable([[0.24923239]])
loss Variable([[0.2491813]])
loss Variable([[0.24913022]])
loss Variable([[0.24907915]])
loss Variable([[0.2490281]])
loss Variable([[0.24897705]])
loss Variable([[0.24892601]])
loss Variable([[0.24887498]])
loss Variable([[0.24882397]])
loss Variable([[0.24877296]])
loss Variable([[0.24872197]])
loss Variable([[0.24867098]])
loss Variable([[0.24862001]])
loss Variable([[0.24856905]])
loss Variable([[0.24851809]])
loss Variable([[0.24846715]])
loss Variable([[0.24841622]])
loss Variable([[0.2483653]])
loss Variable([[0.2483